![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a quad sensor MIPI CSI-2 interface and display the output on a monitor using a DRM/KMS device. This notebook uses the GStreamer multimedia framework. The display device contains a video mixer which allows targeting 4 input video streams onto different video planes with programmable x/y-offsets as well as width and height. 

V4L2 devices that are supported in this notebook:
* MIPI CSI-2 capture pipeline using the Avnet Multi-Camera FMC Module (mipi_quad)

The Avnet Multi-Camera FMC Module (https://www.avnet.com/wps/portal/silica/products/new-products/npi/2018/avnet-multi-camera-fmc-module/) can be used to capture 4 video streams through a MIPI CSI-2 interface. The MIPI CSI-2 capture pipelines are implemented inside the PL and includes a basic ISP.

The video pipeline is composed of the following GStreamer elements:
* The ``mediasrcbin`` element is used to capture video from a V4L2 device. It is a bin element on top of the standard ``v4l2src`` element which performs additional media pipeline initialization (if needed).
* The ``perf`` element is used to measure and print the frame rate
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from Avnet Multi-Camera FMC Module and displays the video on a monitor using a DRM/KMS device.
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, GLib, Gst

This is the Base TRD notebook 5 (nb5).

In [ ]:
nb = "nb5"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create and Configure the GStreamer Elements

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``mipi_quad`` : platform2 only, requires FMC card

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'mipi_quad' : 'vcap_gmsl'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Select the quad-sensor MIPI device as ``source`` based on available media devices for this platform.

In [ ]:
source = "mipi_quad"
media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

The source pads of the ``mediasrcbin`` element are created dynamically when it detects the incoming stream. The ``pad-added`` signal is emitted and this ``pad_added`` callback function is executed. It links the source pads of the mediasrcbin elements to the sink pads of the ``caps`` elements.

Set the ``io-mode`` on all the pads to ``dmabuf``. This will propagate to all the ``v4l2src`` elements inside the bin.

In [ ]:
index = 0
def pad_added(element, pad):
    caps_array = [caps0, caps1, caps2, caps3]
    global index 
    
    sink_pad = caps_array[index].get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)
        pad.set_property("io-mode", "dmabuf")
        
    index = index + 1

Create the ``mediasrcbin`` element which is a bin element that uses the standard ``v4l2src`` element inside. Set the following some properties:
* Set the ``media-device`` property to the desired media device node
* Register the above ``pad_added`` callback function with the ``pad-added`` signal of the ``mediasrcbin`` element.

In [ ]:
src0 = Gst.ElementFactory.make("mediasrcbin")
src0.set_property("media-device", media_device)
src0.connect("pad_added", pad_added);

Create four caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.
The default resolution is set to 960x480 and the format to YUY2.

In [ ]:
width = 960
height = 540
fmt = "YUY2"
framerate = "30/1"

cap = Gst.Caps.from_string("video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt + ", framerate=" + framerate)

caps0 = Gst.ElementFactory.make("capsfilter")
caps0.set_property("caps", cap)

caps1 = Gst.ElementFactory.make("capsfilter")
caps1.set_property("caps", cap)

caps2 = Gst.ElementFactory.make("capsfilter")
caps2.set_property("caps", cap)

caps3 = Gst.ElementFactory.make("capsfilter")
caps3.set_property("caps", cap)

Create the ``perf`` element which is used to measure and print the frame rate while the video pipeline is running.

In [ ]:
perf = Gst.ElementFactory.make("perf")

The display driver creates a DRM device node with the module name ``xlnx``.

List information about the DRM device by passing the module name to the ``modeprint`` utility.

In [ ]:
!modeprint xlnx

Create the ``kmssink`` element and set some properties:
* Set the ``plane-id`` property to the ID value of the target plane. The default value 30 is set to the first RGB plane.
* Set the ``render-rectangle`` property to a quadruple consisting of x-offset, y-offset, width, and height. The render-rectangle allows moving a plane position on the display.

In [ ]:
plane_id0 = 38
plane_id1 = 39
plane_id2 = 40
plane_id3 = 41

render_rectangle0 = Gst.ValueArray((0, 0, width, height))
render_rectangle1 = Gst.ValueArray((width, 0, width, height))
render_rectangle2 = Gst.ValueArray((0, height, width, height))
render_rectangle3 = Gst.ValueArray((width, height, width, height))

sink0 = Gst.ElementFactory.make("kmssink")
sink0.set_property("plane-id", plane_id0)
sink0.set_property("render-rectangle", render_rectangle0)

sink1 = Gst.ElementFactory.make("kmssink")
sink1.set_property("plane-id", plane_id1)
sink1.set_property("render-rectangle", render_rectangle1)

sink2 = Gst.ElementFactory.make("kmssink")
sink2.set_property("plane-id", plane_id2)
sink2.set_property("render-rectangle", render_rectangle2)

sink3 = Gst.ElementFactory.make("kmssink")
sink3.set_property("plane-id", plane_id3)
sink3.set_property("render-rectangle", render_rectangle3)

# 4. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src0)
pipeline.add(caps0)
pipeline.add(sink0)
pipeline.add(caps1)
pipeline.add(sink1)
pipeline.add(caps2)
pipeline.add(sink2)
pipeline.add(caps3)
pipeline.add(perf)
pipeline.add(sink3)

caps0.link(sink0)
caps1.link(sink1)
caps2.link(sink2)
caps3.link(perf)
perf.link(sink3)

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video frames will be displayed on the HDMI monitor connected to the target. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

In [ ]:
pipeline.set_state(Gst.State.PLAYING);
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the GStreamer Pipeline Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from Avnet Multi-Camera FMC Module and display the video on a monitor using a DRM/KMS device.
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>